In [2]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler

In [3]:
from fastapi import FastAPI, HTTPException, Path
from fastapi.responses import HTMLResponse
from pydantic import BaseModel
import pandas as pd
from typing import List, Dict
import os
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

Data Frames Disponibles: <br>
- Nombre de juegos total
- df_juegos_y_genero

In [4]:
users_items_base1 = pd.read_csv('data/users_items_base1.csv')
users_reviews_base1 = pd.read_csv('data/users_reviews_base1.csv')
steam_games_base1 = pd.read_csv('data/steam_games_base1.csv')

In [6]:
nombre_juegos_total = pd.read_csv('df_funciones/listado_juegos_sin_repetidos.csv')

In [7]:
nombre_juegos_total

,item_name,item_id
0,Counter-Strike,10.0
1,Team Fortress Classic,20.0
2,Day of Defeat,30.0
3,Deathmatch Classic,40.0
4,Half-Life: Opposing Force,50.0
...,...,...
10973,NOBUNAGA'S AMBITION: Souzou SengokuRisshiden,375450.0
10974,Alienware Steam Machine,353390.0
10975,ChaosTower,354280.0
10976,Aveyond 4: Shadow Of The Mist,433920.0


In [8]:
col_df_games = ['genres', 'title', 'id', 'developer', 'year']
df_games = steam_games_base1[col_df_games]

df_games

# Renombrar columna
nuevos_nombres = {'id': 'item_id'}
df_games = df_games.rename(columns=nuevos_nombres)

df_games

,genres,title,item_id,developer,year
0,Action,Lost Summoner Kitty,761140,Kotoshiro,2018
1,Casual,Lost Summoner Kitty,761140,Kotoshiro,2018
2,Indie,Lost Summoner Kitty,761140,Kotoshiro,2018
3,Simulation,Lost Summoner Kitty,761140,Kotoshiro,2018
4,Strategy,Lost Summoner Kitty,761140,Kotoshiro,2018
...,...,...,...,...,...
34270,Adventure,Kebab it Up!,745400,Bidoniera Games,2018
34271,Casual,Kebab it Up!,745400,Bidoniera Games,2018
34272,Indie,Kebab it Up!,745400,Bidoniera Games,2018
34273,Casual,EXIT 2 - Directions,658870,"xropi,stev3ns",2017


In [9]:
col_df_item = ['user_id', 'items_count', 'item_id','item_name', 'playtime_forever']
df_items = users_items_base1[col_df_item]

df_items

,user_id,items_count,item_id,item_name,playtime_forever
0,76561197970982479,277,10.0,Counter-Strike,6.0
1,76561197970982479,277,20.0,Team Fortress Classic,0.0
2,76561197970982479,277,30.0,Day of Defeat,7.0
3,76561197970982479,277,40.0,Deathmatch Classic,0.0
4,76561197970982479,277,50.0,Half-Life: Opposing Force,0.0
...,...,...,...,...,...
5113966,76561198329548331,7,373330.0,All Is Dust,0.0
5113967,76561198329548331,7,388490.0,One Way To Die: Steam Edition,3.0
5113968,76561198329548331,7,521570.0,You Have 10 Seconds 2,4.0
5113969,76561198329548331,7,519140.0,Minds Eyes,3.0


In [10]:
df_combinado2 = pd.merge(df_games, df_items, on='item_id', how='inner')

In [11]:
df_combinado2.head()

,genres,title,item_id,developer,year,user_id,items_count,item_name,playtime_forever
0,Action,Carmageddon Max Pack,282010,Stainless Games Ltd,1997,UTNerd24,188,Carmageddon Max Pack,5.0
1,Action,Carmageddon Max Pack,282010,Stainless Games Ltd,1997,I_DID_911_JUST_SAYING,154,Carmageddon Max Pack,0.0
2,Action,Carmageddon Max Pack,282010,Stainless Games Ltd,1997,76561197962104795,79,Carmageddon Max Pack,0.0
3,Action,Carmageddon Max Pack,282010,Stainless Games Ltd,1997,r3ap3r78,331,Carmageddon Max Pack,0.0
4,Action,Carmageddon Max Pack,282010,Stainless Games Ltd,1997,saint556,583,Carmageddon Max Pack,13.0


In [12]:
df_combinado2_filtrado = df_combinado2.drop(columns=['item_name', 'year','user_id', 'items_count'])

In [13]:
df_recomend_juego_fc6 = df_combinado2_filtrado.copy()

In [15]:
# df_recomend_juego_fc6.to_csv("df_funciones/df_recomend_juego_fc6.csv", index=False)

In [16]:
df_combinado2_filtrado

,genres,title,item_id,developer,playtime_forever
0,Action,Carmageddon Max Pack,282010,Stainless Games Ltd,5.0
1,Action,Carmageddon Max Pack,282010,Stainless Games Ltd,0.0
2,Action,Carmageddon Max Pack,282010,Stainless Games Ltd,0.0
3,Action,Carmageddon Max Pack,282010,Stainless Games Ltd,0.0
4,Action,Carmageddon Max Pack,282010,Stainless Games Ltd,13.0
...,...,...,...,...,...
9341584,Action,Counter-Strike: Condition Zero,80,Valve,0.0
9341585,Action,Counter-Strike: Condition Zero,80,Valve,0.0
9341586,Action,Counter-Strike: Condition Zero,80,Valve,0.0
9341587,Action,Counter-Strike: Condition Zero,80,Valve,9.0


## FUNCION QUE CORRE BIEN! <BR>
COMPARA TITULOS Y GENEROS

Funcion que anda bien pero no carga tampoco en la API

In [17]:
# @app.get('/RecomendacionJuego/{id_juego}')
def recomendacion_juego(id_juego: int, df_entrada = df_combinado2_filtrado ):
    '''
    Endpoint para obtener una lista de juegos recomendados similares a un juego dado.

    Parámetros:
    - id_juego (int): ID del juego para el cual se desean obtener recomendaciones.

    Respuestas:
    - 200 OK: Retorna una lista con 5 juegos recomendados similares al juego ingresado.
    - 404 Not Found: Si no se encuentra el juego con el ID especificado.
    - 500 Internal Server Error: En caso de cualquier otro error, proporciona detalles de la excepción.

    Ejemplo de Uso:
    - /RecomendarJuego/123

    Ejemplo de Respuesta Exitosa:
    [
        {"title": "Juego A"},
        {"title": "Juego B"},
        {"title": "Juego C"},
        {"title": "Juego D"},
        {"title": "Juego E"}
    ]
    '''
    try:
        # Busca el juego en el DataFrame por ID
        juego_seleccionado = df_entrada[df_entrada['item_id'] == id_juego]

        # Verifica si el juego con el ID especificado existe
        if juego_seleccionado.empty:
            raise HTTPException(status_code=404, detail=f"No se encontró el juego con ID {id_juego}")

        # Genera una variable con el titulo y genero en tipo str
        title_game_and_genres = ' '.join(juego_seleccionado['title'].fillna('').astype(str) + ' ' + juego_seleccionado['genres'].fillna('').astype(str))
        
        # Genera una cadena de texto con los titulos y los generos
        chunk_tags_and_genres = df_entrada['title'].fillna('').astype(str) + ' ' + df_entrada['genres'].fillna('').astype(str)
        
        # Genera una lista con dos elementos, el juego cargado y el resto de los juegos a comparar
        games_to_compare = [title_game_and_genres] + chunk_tags_and_genres.tolist()

        tfidf_vectorizer = TfidfVectorizer(min_df=5, max_df=0.9)
        tfidf_matrix = tfidf_vectorizer.fit_transform(games_to_compare)

        # Generamos una limitacion truncando datos por un gran exceso de datos
        # Selecciona las primeras 15000 características
        tfidf_matrix_subset = tfidf_matrix[:15000, :]

        # Ahora a esos datos truncados toma los mejores 1000 campos y los compara con el dato de entrada
        svd = TruncatedSVD(n_components=1000)
        tfidf_matrix_reduced = svd.fit_transform(tfidf_matrix_subset)

        # Calcula la similiridad de cosine
        # Calcula la matriz de similitud entre juego de entrada y resto de los juegos
        similarity_scores = cosine_similarity(tfidf_matrix_reduced)

        if similarity_scores is not None:
            # Ordena los indices similares de forma descendente y se toma desde el segundo para tomar los mas similares
            similar_games_indices = similarity_scores[0].argsort()[::-1]

            # Tomar las mejores 5 recomendaciones sin incluir el juego seleccionado
            num_recommendations = 5
            recommended_games = df_entrada.loc[similar_games_indices[1:num_recommendations + 1]]
            unique_recommendations = set()
            filtered_recommendations = []

            for index in similar_games_indices[1:]:
                game = df_entrada.loc[index, 'title']
                if game not in unique_recommendations:
                    unique_recommendations.add(game)
                    filtered_recommendations.append(game)

                if len(filtered_recommendations) >= num_recommendations:
                    break

            # Devolver la lista de juegos recomendados
            recommended_games = [{"title": game} for game in filtered_recommendations]

            return recommended_games

        return {"message": "No se encontraron juegos similares."}

    except Exception as e:
        return {"message": f"Error: {str(e)}"}

In [18]:
respuesta = recomendacion_juego(10)
respuesta

c:\Users\Usuario\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\extmath.py:192: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


[{'title': 'Half-Life'},
 {'title': "Disciples II: Gallean's Return"},
 {'title': 'The Ship: Murder Party'},
 {'title': 'Carmageddon Max Pack'},
 {'title': 'Disciples II: Rise of the Elves'}]

----
---
---
# Analisis de Funcion paso a paso

In [19]:
df_entrada = df_combinado2_filtrado
id_juego = 10

In [20]:
# Busca el juego en el DataFrame por ID
juego_seleccionado = df_entrada[df_entrada['item_id'] == id_juego]

In [21]:
juego_seleccionado_1 = juego_seleccionado.head(1)

In [22]:
juego_seleccionado_1

,genres,title,item_id,developer,playtime_forever
9306484,Action,Counter-Strike,10,Valve,6.0


In [23]:
title_game_and_genres = ' '.join(juego_seleccionado_1['title'].fillna('').astype(str) + ' ' + juego_seleccionado_1['genres'].fillna('').astype(str))

In [24]:
title_game_and_genres

'Counter-Strike Action'

In [25]:
tfidf_vectorizer = TfidfVectorizer()

In [26]:
chunk_size = 100   
similarity_scores = None

In [27]:
chunk_tags_and_genres = df_entrada['title'].fillna('').astype(str) + ' ' + df_entrada['genres'].fillna('').astype(str)
chunk_tags_and_genres

0                    Carmageddon Max Pack Action
1                    Carmageddon Max Pack Action
2                    Carmageddon Max Pack Action
3                    Carmageddon Max Pack Action
4                    Carmageddon Max Pack Action
                           ...                  
9341584    Counter-Strike: Condition Zero Action
9341585    Counter-Strike: Condition Zero Action
9341586    Counter-Strike: Condition Zero Action
9341587    Counter-Strike: Condition Zero Action
9341588    Counter-Strike: Condition Zero Action
Length: 9341589, dtype: object

In [28]:
games_to_compare = [title_game_and_genres] + chunk_tags_and_genres.tolist()
games_to_compare

# Mas campos repetidos cosine considera ese juego con su genero con mas peso en la comparacion

['Counter-Strike Action',
 'Carmageddon Max Pack Action',
 'Carmageddon Max Pack Action',
 'Carmageddon Max Pack Action',
 'Carmageddon Max Pack Action',
 'Carmageddon Max Pack Action',
 'Carmageddon Max Pack Action',
 'Carmageddon Max Pack Action',
 'Carmageddon Max Pack Action',
 'Carmageddon Max Pack Action',
 'Carmageddon Max Pack Action',
 'Carmageddon Max Pack Action',
 'Carmageddon Max Pack Action',
 'Carmageddon Max Pack Action',
 'Carmageddon Max Pack Action',
 'Carmageddon Max Pack Action',
 'Carmageddon Max Pack Action',
 'Carmageddon Max Pack Action',
 'Carmageddon Max Pack Action',
 'Carmageddon Max Pack Action',
 'Carmageddon Max Pack Action',
 'Carmageddon Max Pack Action',
 'Carmageddon Max Pack Action',
 'Carmageddon Max Pack Action',
 'Carmageddon Max Pack Action',
 'Carmageddon Max Pack Action',
 'Carmageddon Max Pack Action',
 'Carmageddon Max Pack Action',
 'Carmageddon Max Pack Action',
 'Carmageddon Max Pack Action',
 'Carmageddon Max Pack Action',
 'Carmageddon 

In [29]:
tfidf_matrix = tfidf_vectorizer.fit_transform(games_to_compare)

Filtrar Términos Poco Frecuentes o Demasiado Frecuentes:

Puedes ajustar los parámetros del vectorizador TF-IDF para filtrar términos que aparecen muy raramente o demasiado frecuentemente. Esto podría reducir el tamaño de la matriz y eliminar términos que no contribuyen significativamente a la similitud entre los juegos.

min_df: Ignora los términos que aparecen en menos de min_df documentos.
max_df: Ignora los términos que aparecen en más del max_df porcentaje de documentos.

In [30]:
tfidf_vectorizer = TfidfVectorizer(min_df=5, max_df=0.9)
tfidf_matrix = tfidf_vectorizer.fit_transform(games_to_compare)


In [ ]:
# Generamos una limitacion truncando datos por un gran exceso de datos
# Por ejemplo, selecciona las primeras 500 características
tfidf_matrix_subset = tfidf_matrix[:15000, :]


In [ ]:
# from sklearn.decomposition import TruncatedSVD

# Ahora a esos datos truncados toma los mejores 300 campos y los compara
svd = TruncatedSVD(n_components=1000)
tfidf_matrix_reduced = svd.fit_transform(tfidf_matrix_subset)

In [ ]:
similarity_scores = cosine_similarity(tfidf_matrix_reduced)

c:\Users\Usuario\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\extmath.py:192: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


In [ ]:
# Ordena los indices similares de forma descendente y se toma desde el segundo para tomar los mas similares
similar_games_indices = similarity_scores[0].argsort()[::-1]

In [ ]:
similar_games_indices

array([   0, 5326, 5352, ...,  460,  459,  349], dtype=int64)

In [ ]:
num_recommendations = 5
recommended_games = df_entrada.loc[similar_games_indices[1:num_recommendations + 1]]

In [ ]:
recommended_games

,genres,title,item_id,developer,playtime_forever
5326,Action,Half-Life,70,Valve,1582.0
5352,Action,Half-Life,70,Valve,96.0
5351,Action,Half-Life,70,Valve,561.0
5350,Action,Half-Life,70,Valve,0.0
5349,Action,Half-Life,70,Valve,321.0


In [ ]:
# Tomar las mejores 5 recomendaciones sin incluir el juego seleccionado
unique_recommendations = set()
filtered_recommendations = []

for index in similar_games_indices[1:]:
    game = df_entrada.loc[index, 'title']
    if game not in unique_recommendations:
        unique_recommendations.add(game)
        filtered_recommendations.append(game)

    if len(filtered_recommendations) >= num_recommendations:
        break

# Devolver la lista de juegos recomendados
recommended_games = [{"title": game} for game in filtered_recommendations]


In [ ]:
recommended_games

[{'title': 'Half-Life'},
 {'title': "Disciples II: Gallean's Return"},
 {'title': 'The Ship: Murder Party'},
 {'title': 'Carmageddon Max Pack'},
 {'title': 'Disciples II: Rise of the Elves'}]

-----
----
----
----
----
---

----
-----
INTENTO DE AGREGAR TIME PLAY

Mejor función generada pero tampoco funciona en la API

In [ ]:
# @app.get('/RecomendacionJuego/{id_juego}')
def recomendacion_juego_conplaytime(id_juego: int,df_entrada = df_combinado2_filtrado ):
    '''
    Endpoint para obtener una lista de juegos recomendados similares a un juego dado.

    Parámetros:
    - id_juego (int): ID del juego para el cual se desean obtener recomendaciones.

    Respuestas:
    - 200 OK: Retorna una lista con 5 juegos recomendados similares al juego ingresado.
    - 404 Not Found: Si no se encuentra el juego con el ID especificado.
    - 500 Internal Server Error: En caso de cualquier otro error, proporciona detalles de la excepción.

    Ejemplo de Uso:
    - /RecomendarJuego/123

    Ejemplo de Respuesta Exitosa:
    [
        {"title": "Juego A"},
        {"title": "Juego B"},
        {"title": "Juego C"},
        {"title": "Juego D"},
        {"title": "Juego E"}
    ]
    '''
    try:
        # Busca el juego en el DataFrame por ID
        juego_seleccionado = df_entrada[df_entrada['item_id'] == id_juego]

        # Verifica si el juego con el ID especificado existe
        if juego_seleccionado.empty:
            raise HTTPException(status_code=404, detail=f"No se encontró el juego con ID {id_juego}")

    

        # Combina título, géneros y tiempo jugado del juego seleccionado en un solo texto
        title_game_genres_playtime = ' '.join(
            juego_seleccionado['title'].fillna('').astype(str) + ' ' +
            juego_seleccionado['genres'].fillna('').astype(str) + ' ' +
            juego_seleccionado['playtime_forever'].fillna('').astype(str)
        )

        # Información de títulos, géneros y tiempo jugado de todos los juegos
        chunk_tags_genres_playtime = (
            df_entrada['title'].fillna('').astype(str) + ' ' +
            df_entrada['genres'].fillna('').astype(str) + ' ' +
            df_entrada['playtime_forever'].fillna('').astype(str)
        )

        # Genera una cadena de texto con los titulos y los generos
        chunk_tags_genres_playtime = df_entrada['title'].fillna('').astype(str) + ' ' + df_entrada['genres'].fillna('').astype(str)
        
        # Genera una lista con dos elementos, el juego cargado y el resto de los juegos a comparar
        games_to_compare = [title_game_genres_playtime] + chunk_tags_genres_playtime.tolist()

        tfidf_vectorizer = TfidfVectorizer(min_df=5, max_df=0.9)
        tfidf_matrix = tfidf_vectorizer.fit_transform(games_to_compare)


        # Generamos una limitacion truncando datos por un gran exceso de datos
        # Selecciona las primeras 15000 características
        tfidf_matrix_subset = tfidf_matrix[:10000, :]

        # Ahora a esos datos truncados toma los mejores 1000 campos y los compara con el dato de entrada
        svd = TruncatedSVD(n_components=1000)
        tfidf_matrix_reduced = svd.fit_transform(tfidf_matrix_subset)

        # Calcula la similiridad de cosine
        # Calcula la matriz de similitud entre juego de entrada y resto de los juegos
        similarity_scores = cosine_similarity(tfidf_matrix_reduced)

        if similarity_scores is not None:
            # Ordena los indices similares de forma descendente y se toma desde el segundo para tomar los mas similares
            similar_games_indices = similarity_scores[0].argsort()[::-1]

            # Tomar las mejores 5 recomendaciones sin incluir el juego seleccionado
            num_recommendations = 5
            recommended_games = df_entrada.loc[similar_games_indices[1:num_recommendations + 1]]
            unique_recommendations = set()
            filtered_recommendations = []

            for index in similar_games_indices[1:]:
                game = df_entrada.loc[index, 'title']
                if game not in unique_recommendations:
                    unique_recommendations.add(game)
                    filtered_recommendations.append(game)

                if len(filtered_recommendations) >= num_recommendations:
                    break

            # Devolver la lista de juegos recomendados
            recommended_games = [{"title": game} for game in filtered_recommendations]

            return recommended_games

        return {"message": "No se encontraron juegos similares."}

    except Exception as e:
        return {"message": f"Error: {str(e)}"}

In [ ]:
id_juego_ejemplo = 10
resultados_conplay = recomendacion_juego_conplaytime(id_juego_ejemplo)
resultados_conplay

[{'title': 'Half-Life'},
 {'title': "Disciples II: Gallean's Return"},
 {'title': 'The Ship: Murder Party'},
 {'title': 'Carmageddon Max Pack'},
 {'title': 'Disciples II: Rise of the Elves'}]

In [ ]:
id_juego_ejemplo_B = 30
resultados_conplay_B = recomendacion_juego_conplaytime(id_juego_ejemplo_B)
resultados_conplay_B

[{'title': 'Half-Life'},
 {'title': "Disciples II: Gallean's Return"},
 {'title': 'Disciples II: Rise of the Elves'},
 {'title': 'The Ship: Murder Party'},
 {'title': 'Carmageddon Max Pack'}]

In [ ]:
id_juego_ejemploA = 40
resultados_conplay_A = recomendacion_juego_conplaytime(id_juego_ejemploA)
resultados_conplay_A

[{'title': 'Half-Life'},
 {'title': "Disciples II: Gallean's Return"},
 {'title': 'The Ship: Murder Party'},
 {'title': 'Carmageddon Max Pack'},
 {'title': 'Disciples II: Rise of the Elves'}]

Agregando PlayTime_Forever se pudo ver una gran mejora en tiempo de ejecucion, se ve que ayuda a elegir mejor.